In [12]:
import pandas
import numpy
import scipy
import string
import pprint
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import TweetTokenizer
from porter2stemmer import Porter2Stemmer
import nltk
from nltk.corpus import stopwords

import gensim
import os
import collections
import smart_open
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

tknzr = TweetTokenizer()
stemmer = Porter2Stemmer()
stopWords = set(stopwords.words('english'))
TOKENS = []
def readfile(fname):
    array = []
    file = open(fname, "r")
    lines = file.readlines()
    filestring = " ".join(lines)
    everything = ' '.join(filestring.split(' '))
    tokens = tknzr.tokenize(everything);
    stemmedtokens = stem(tokens)
    global TOKENS
    TOKENS = stemmedtokens
    return TOKENS

#This function takes an array and iterates over each word in it.
#If the word is not in stopwords, it stems it and appends it to a new array.
def stem(array):
    stemmed = []
    for w in array:
        if w not in stopWords:
            w = stemmer.stem(w)
            stemmed.append(w)
    return stemmed
    


In [13]:
mtarr = readfile("./marktwain/combined.txt")

In [14]:
mtstemmedwords = stem(mtarr)
len(mtstemmedwords)

254311

In [15]:
jaarr = readfile('./janeausten/ja_COMBINED_treated.txt')
jastemmedwords = stem(jaarr)
len(jastemmedwords)

292656

In [16]:
import numpy as np
jaChunksArr = np.array_split(jastemmedwords, 10000)
mtChunksArr = np.array_split(mtstemmedwords, 10000)

In [17]:
from gensim.models import doc2vec
mtStemmedArray = []
prefix = 'marktwain'
for item_no,item in enumerate(mtChunksArr):
    global mtStemmedArray
    item = [item.tolist(),[prefix+"_%s" % item_no]]
    mtStemmedArray.append(item)
sentenceStemmedArray = []
for item in mtStemmedArray:
    sentence = doc2vec.LabeledSentence(item[0], item[1])
    sentenceStemmedArray.append(sentence)
sentenceStemmedArray

[LabeledSentence(words=['connecticut', 'yank', 'king', 'arthur', 'court', 'word', 'explan', 'warwick', 'castl', 'came', 'across', 'curious', 'strang', 'go', 'talk', 'attract', 'thr', 'thing', 'candid', 'simplic', 'marvel', 'familiar', 'ancient', 'armor', 'rest', 'compani'], tags=['marktwain_0']),
 LabeledSentence(words=['talk', 'fell', 'togeth', 'modest', 'peopl', 'tail', 'herd', 'shown', 'began', 'say', 'thing', 'interest', 'talk', 'along', 'soft', 'pleasant', 'flow', 'seem', 'drift', 'away', 'impercept', 'world', 'time', 'remot', 'era', 'old'], tags=['marktwain_1']),
 LabeledSentence(words=['forgotten', 'countri', 'gradual', 'wove', 'spell', 'seem', 'move', 'among', 'spect', 'shadow', 'dust', 'mold', 'gray', 'antiqu', 'hold', 'speech', 'relic', 'exact', 'would', 'speak', 'nearest', 'person', 'friend', 'enemi', 'familiar', 'neighbor'], tags=['marktwain_2']),
 LabeledSentence(words=['spoke', 'sir', 'bediv', 'sir', 'bor', 'de', 'gani', 'sir', 'launcelot', 'lake', 'sir', 'galahad', 'grea

In [18]:
jaStemmedArray = []
prefix = 'janeausten'
for item_no,item in enumerate(jaChunksArr):
    global jaStemmedArray
    item = [item.tolist(),[prefix+"_%s" % item_no]]
    jaStemmedArray.append(item)
for item in jaStemmedArray:
    sentence = doc2vec.LabeledSentence(item[0],item[1])
    sentenceStemmedArray.append(sentence)

In [19]:
len(sentenceStemmedArray)

20000

In [20]:
combinedStemmedModel = doc2vec.Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
combinedStemmedModel.build_vocab(sentenceStemmedArray)

In [37]:
from random import shuffle
for epoch in range(10):
    combinedStemmedModel.train(sentenceStemmedArray, total_examples=combinedStemmedModel.corpus_count, epochs=combinedStemmedModel.iter)
    shuffle(sentenceStemmedArray)
    combinedStemmedModel.alpha -= 0.002  # decrease the learning rate
    combinedStemmedModel.min_alpha = combinedStemmedModel.alpha  # fix the learning rate, no decay

In [38]:
train = np.zeros((20000,100))
for i in range(20000):
    train[i] = combinedStemmedModel.docvecs[i]

In [39]:
#this is wrong
# stemmedlist = []
# for doc in sentenceStemmedArray:
#     iv = combinedStemmedModel.infer_vector(doc.words)
#     stemmedlist.append(iv)
# len(stemmedlist)

In [40]:
import pandas as pd
stemmedX = pd.DataFrame(train)
stemmedX

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.177390,-0.023978,0.149622,0.069912,0.116786,-0.444700,0.011870,-0.087726,0.362657,0.248551,...,0.453428,0.069737,-0.037616,0.129814,0.052803,0.042477,0.028521,0.326419,0.183162,-0.411591
1,0.015139,0.228541,0.244686,-0.010413,-0.110800,0.041041,0.320123,0.260960,0.315335,0.321989,...,0.398590,-0.022607,-0.267130,-0.024783,0.019888,-0.312057,-0.481003,0.027718,0.246420,-0.076467
2,0.324050,0.023148,0.006460,0.014187,-0.114111,-0.036989,0.208511,-0.031280,0.384838,0.776529,...,0.836708,-0.114369,-0.761796,0.053665,0.182113,-0.000038,-0.430895,-0.068272,0.370685,-0.314235
3,-0.022448,0.443219,0.173263,0.347991,-0.348206,-0.077723,-0.226454,-0.103876,0.476259,0.000105,...,0.456661,0.164794,-0.196436,-0.074305,0.029605,-0.075560,0.098831,0.178173,-0.008123,0.100175
4,0.132747,-0.014109,0.443995,-0.171937,-0.129884,-0.171969,0.119834,0.000888,0.258262,0.003681,...,0.227917,-0.338870,-0.292604,-0.367974,-0.100213,-0.230086,-0.210244,-0.123118,0.058613,-0.032143
5,-0.189430,0.372314,-0.328672,0.149694,-0.236780,-0.240267,0.070847,-0.149775,-0.087507,0.199626,...,0.305370,0.065746,-0.027427,-0.039029,0.081325,0.178498,0.141087,0.131031,0.092560,-0.129581
6,0.049359,0.134286,0.217897,0.350526,0.092141,-0.181885,-0.437692,-0.049287,0.294604,0.161419,...,0.029165,0.338725,-0.098311,0.158110,0.035244,0.434647,-0.146981,0.107397,-0.249890,0.107200
7,-0.150949,-0.086482,-0.001879,0.013788,0.271148,-0.095534,0.474041,-0.104157,0.065312,-0.089426,...,0.579631,0.044853,-0.099905,0.098389,0.402778,0.274578,0.240704,-0.132566,0.090368,-0.007736
8,0.334682,0.321740,0.154964,0.245857,-0.011907,-0.194965,-0.014267,0.121400,0.372935,-0.081425,...,0.261696,0.107092,-0.436436,0.502222,0.234845,0.042861,-0.331018,0.058383,-0.113875,0.146649
9,0.162213,0.270266,0.059372,0.173021,-0.052907,-0.369646,0.440764,0.206514,0.024097,0.560404,...,0.508595,0.195833,-0.279761,0.190269,0.291729,0.383236,-0.027365,0.195286,0.193701,-0.046204


In [41]:
yzero = np.ones(10000)
yone = np.zeros(10000)
yzero = pd.DataFrame(yzero)
yone = pd.DataFrame(yone)
Y = pd.concat([yone,yzero])
len(Y)

20000

In [42]:
from sklearn.model_selection import train_test_split
X_train_stemmed, X_test_stemmed, y_train_stemmed, y_test_stemmed = train_test_split(stemmedX, Y, test_size=0.25)
print (X_train_stemmed.shape, y_train_stemmed.shape)
print (X_test_stemmed.shape, y_test_stemmed.shape)

(15000, 100) (15000, 1)
(5000, 100) (5000, 1)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
rbf = SVC(C=1.0, kernel='rbf')
rbf.fit(X_train_stemmed, y_train_stemmed)
rbfStemmedPredictions = rbf.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, rbfStemmedPredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 88.70%


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
poly = SVC(C=1.0, kernel='poly',degree=5)
poly.fit(X_train_stemmed, y_train_stemmed)
polyStemmedPredictions = poly.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, polyStemmedPredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=50, random_state=0)
rfc.fit(X_train_stemmed, y_train_stemmed)
rfcpredictions = rfc.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, rfcpredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Accuracy: 84.92%


In [30]:
#logistic regression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression()
logreg.fit(X_train_stemmed, y_train_stemmed)
logregpredictions = logreg.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, logregpredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 88.16%


In [31]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation='relu', solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(X_train_stemmed, y_train_stemmed)
mlppredictions = mlp.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, mlppredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 89.70%


In [3]:
#Testing on untrained books:
mtarr = readfile("./marktwain/marktwain_roughingit.txt")